In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.approximated_controller_full_feasible_sets import ApproximatedHybridModelPredictiveController, PolicySampler, upload_PolicySampler
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt

# Construction of Box-Atlas

Robot's limbs

In [3]:
from pympc.models.boxatlas_parameters import dynamics
penetration = dynamics['mass'] * dynamics['gravity'] / dynamics['normal_stiffness']

# right hand
A_rh = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[1., 0.],[-1., 0.],[0., -1.]]),
}
b_rh = {
    'not_in_contact': np.array([[.4],[.5]]),
    'in_contact': np.array([[-.4],[.4+penetration],[.5]]),
}
contact_surfaces_rh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rh = np.array([[-.3], [.0]])
parametric_wall_distance = {
    'label': 'wall_distance_rh',
    'not_in_contact': {'index': [0], 'coefficient': [1.]},
    'in_contact': {'index': [0, 1], 'coefficient': [-1., 1.]},
    'min': -.05,
    'max': .05,
} # if positive moves the wall away from the body
parameters_rh = [parametric_wall_distance]
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh, parameters=parameters_rh)

# left foot
A_lf = {
    'not_in_contact': np.array([[0., -1.]]),
    'in_contact': np.array([[0., 1.],[0., -1.]]),
}
b_lf = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+2.*penetration]]),
}
contact_surfaces_lf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lf = np.array([[.2], [-.5 - penetration / 2.]])
lf = MovingLimb(A_lf, b_lf, contact_surfaces_lf, q_lf)

# right foot
A_rf = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[0., 1.],[-1., 0.],[0., -1.]]),
}
b_rf = {
    'not_in_contact': np.array([[.5],[.5]]),
    'in_contact': np.array([[-.5],[.5],[.5+2.*penetration]]),
}
contact_surfaces_rf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rf = np.array([[-.2], [-.5 - penetration / 2.]])
rf = MovingLimb(A_rf, b_rf, contact_surfaces_rf, q_rf)

Assemble robot

In [4]:
limbs = { 'moving': {'rh': rh, 'lf': lf, 'rf': rf}, 'fixed': {} }
nominal_mode = {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
forbidden_modes = [{'lf': 'not_in_contact', 'rf': 'not_in_contact'}]
box_atlas = BoxAtlas(limbs, nominal_mode, forbidden_modes)

Computation of Maximal Invariant Constraint-Admissible Set started.
Determinedness index: 6, Convergence index: 0.0, Number of facets: 231.                             
Maximal Invariant Constraint-Admissible Set found.
Removing redundant facets ... minimal facets are 77.
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Parameter FeasibilityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter NumericFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0
Parameter OptimalityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter MIPGap unchanged
   Value: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Parameter IntFeasTol unchanged
   Value: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Parameter MIPFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0


Print state, input, and modes

In [5]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
box_atlas.print_modes()


Box-Atlas states:
0: wall_distance_rh
1: qlfx
2: qlfy
3: qrfx
4: qrfy
5: qrhx
6: qrhy
7: qbx
8: qby
9: tb
10: vbx
11: vby
12: ob

Box-Atlas inputs:
0: vlfx
1: vlfy
2: vrfx
3: vrfy
4: vrhx
5: vrhy

Box-atlas modes:
0: {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
1: {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}
2: {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'not_in_contact'}
3: {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'in_contact'}
4: {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
5: {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}


# Hybrid MPC

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, x, ss = box_atlas.controller.feedforward(x_0)[0:3]

In [ ]:
prog = box_atlas.controller.condense_program(ss)
print prog.C_u.shape

Closed-loop system simulation of the MPC controller

In [ ]:
N_sim = 50
u = []
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[1,0] = .15
x_0[3,0] = .15
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
nominal_mode = box_atlas.get_mode_index(box_atlas.nominal_mode)
for k in range(N_sim):
    u_k, x_k, ss_k = box_atlas.controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print 'Time step ' + str(k) + ': ' + str(ss_k) + '.'
    u_ws = u_k[1:] + [box_atlas.K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (nominal_mode,)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    # print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)
# traj_miqp = Trajectory(x, u, box_atlas.Q, box_atlas.R, box_atlas.P)
# np.save('box_atlas_tajectory_miqp_lh', traj_miqp)

Visualization of the simulation with Director

In [ ]:
from pympc.models.boxatlas_parameters import dynamics
box_atlas.visualize(x[0])
time.sleep(10 * dynamics['sampling_time'])
for x_t in x:
    box_atlas.visualize(x_t)
    time.sleep(dynamics['sampling_time'])

In [ ]:
box_atlas.print_mode_sequence(ss)

# Approximate Hybrid MPC

Initialization of the library

In [6]:
sampler = PolicySampler(box_atlas.controller)
samples_so_far = 14600
if samples_so_far > 0:
    sampler = upload_PolicySampler('sampler_box_atlas_' + str(samples_so_far))

Parameter FeasibilityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter NumericFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0
Parameter OptimalityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06


Coverage

In [ ]:
sample_step = 100
n_steps = 100
for i in range(n_steps):
    n = samples_so_far + (i+1)*sample_step
    if samples_so_far == 0 and i == 0:
        sample_origin = True
    else:
        sample_origin = False
    sampler.sample_policy(sample_step, box_atlas.is_inside_a_domain, sample_origin)
    print(str(n) + ' samples added to the library.')
    sampler.save('sampler_box_atlas_' + str(n))

Synthesize approximated contoller

In [7]:
approximated_controller = ApproximatedHybridModelPredictiveController(sampler, box_atlas.get_mode_index(nominal_mode))

In [8]:
approximated_controller.tree.plot('box_atlas_tree')

In [9]:
len(sampler.qp_library)

98

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, ms = approximated_controller.feedback(x_0)

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, x, ss = box_atlas.controller.feedforward(x_0)[0:3]

Closed-loop system simulation of the approximate MPC controller

In [28]:
N_sim = 50
u = []
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[10,0] = 1.5
x = [x_0]
ms_ws = None
for k in range(N_sim):
    print '\nTime step ' + str(k) + ':'
    u_k, ms_ws = approximated_controller.feedback(x[k], ms_ws)
    ms_ws = ms_ws[1:] + (box_atlas.get_mode_index(nominal_mode),)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k])[0][1]
    u.append(u_k)
    x.append(x_next)


Time step 0:
No warm start provided.
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Checking mode sequence fragment (4,):
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    infeasible, branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    infeasible, branch cut.
Checking mode sequence fragment (4, 0):
    further branch.
Checking mode sequence fragment (4, 0, 4, 2, 2, 2, 2, 0, 0, 0):
    new upper bound with cost 9.35310575832.
Checking mode sequence fragment (4, 0, 2, 5, 3, 3, 3, 3, 3, 5):
    infeasible, branch cut.
Checking mode sequence fragment (4, 0, 1, 1, 0, 4, 3, 3, 3, 3):
    infeasible, branch cut.
Checking mode sequence fragment (4, 1):
    infeasible, branch cut.
Checking mode sequence fragment (4, 2):
    further branch.
Checking mode sequence fragment (4, 2, 1, 1, 0, 4, 3, 3, 3, 3):
    infeasible, branch cut.
Checking mode sequence fragment (4, 2, 2, 2, 2, 4):
    further branch.
Checking 

In [30]:
from pympc.models.boxatlas_parameters import dynamics
box_atlas.visualize(x[0])
time.sleep(10 * dynamics['sampling_time'])
for x_t in x:
    box_atlas.visualize(x_t)
    time.sleep(dynamics['sampling_time'])